| Concept           | Frequency Range   | Notes / MIDI Range      | Notes                        |
| ----------------- | ----------------- | ----------------------- | ---------------------------- |
| Audible (human)   | 20 Hz – 20 kHz    | MIDI 0 – 127            | Full MIDI                    |
| Playful / Musical | 27.5 Hz – 4186 Hz | MIDI 21 (A0) – 108 (C8) | Piano & general instruments  |
| Safe / Harmonics  | 80 Hz – 12 kHz    | MIDI ~28 – 110          | Most instruments and effects |


In [2]:
print("setup seccessful")

setup seccessful


### MIDI Note Numbers to Frequency

The formula to convert a MIDI note number to frequency (Hz) is:

$$
f = 440 \times 2^{\frac{(n - 69)}{12}}
$$

Where:
- \( f \) = frequency in Hz
- \( n \) = MIDI note number
- 440 Hz = standard tuning for A4 (MIDI note 69)

**Example:**
- Middle C (MIDI 60):
$$
f = 440 \times 2^{\frac{60 - 69}{12}} \approx 261.63 \text{ Hz}
$$

---

### Frequency to MIDI Note Number

To find the MIDI note number from a frequency:

$$
n = 69 + 12 \times \log_2\left(\frac{f}{440}\right)
$$

**Example:**
- Frequency 523.25 Hz (C5):
$$
n = 69 + 12 \times \log_2\left(\frac{523.25}{440}\right) \approx 72
$$

---

### MIDI Note Number to Musical Note

A common mapping for note names:

```text
0 = C-1
1 = C#-1 / Db-1
2 = D-1
...
60 = C4 (Middle C)
69 = A4


In [3]:
# List of note names in an octave
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

print(f"{'MIDI':<5} {'Note':<4}")
print('-' * 10)

# MIDI note numbers typically range from 0 to 127
for midi_num in range(69,128):
    octave = (midi_num // 12) - 1      # MIDI standard: C4 = 60
    note = note_names[midi_num % 12]
    print(f"{midi_num:<5} {note}{octave:<3}")


MIDI  Note
----------
69    A4  
70    A#4  
71    B4  
72    C5  
73    C#5  
74    D5  
75    D#5  
76    E5  
77    F5  
78    F#5  
79    G5  
80    G#5  
81    A5  
82    A#5  
83    B5  
84    C6  
85    C#6  
86    D6  
87    D#6  
88    E6  
89    F6  
90    F#6  
91    G6  
92    G#6  
93    A6  
94    A#6  
95    B6  
96    C7  
97    C#7  
98    D7  
99    D#7  
100   E7  
101   F7  
102   F#7  
103   G7  
104   G#7  
105   A7  
106   A#7  
107   B7  
108   C8  
109   C#8  
110   D8  
111   D#8  
112   E8  
113   F8  
114   F#8  
115   G8  
116   G#8  
117   A8  
118   A#8  
119   B8  
120   C9  
121   C#9  
122   D9  
123   D#9  
124   E9  
125   F9  
126   F#9  
127   G9  


In [6]:
!pip install mido pygame

  Using cached mido-1.3.3-py3-none-any.whl.metadata (6.4 kB)
  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached mido-1.3.3-py3-none-any.whl (54 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/10.6 MB 5.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/10.6 MB 7.1 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/10.6 MB 6.6 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/10.6 MB 5.8 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/10.6 MB 4.8 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.6 MB 3.9 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.6 MB 3.9 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/10.6 MB 3.2 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/10.6 MB 3.2 MB/


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from mido import Message, MidiFile, MidiTrack


## 1. Beats

In [9]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

KICK = 36; SNARE = 38; HIHAT = 42
ticks = 480

pattern = [
    (KICK, 0),
    (HIHAT, 0),
    (SNARE, ticks),
    (HIHAT, 0),
    (KICK, ticks),
    (HIHAT, 0),
    (SNARE, ticks),
    (HIHAT, 0)
]

for note, delay in pattern:
    track.append(Message('note_on', note=note, velocity=100, time=delay))
    track.append(Message('note_off', note=note, velocity=100, time=120))

mid.save("midi-clips/beat.mid")


## 2. Harmony Clip (Chord Progression)

In [10]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

chords = [
    [60,64,67],   # C
    [67,71,74],   # G
    [69,72,76],   # Am
    [65,69,72],   # F
]

for chord in chords:
    for n in chord:
        track.append(Message('note_on', note=n, velocity=80, time=0))
    for n in chord:
        track.append(Message('note_off', note=n, velocity=80, time=960))

mid.save("midi-clips/harmony.mid")


## 3. Melody Clip (Long Single Clip)

In [11]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

melody_notes = [60, 62, 64, 67, 69, 67, 64, 62] * 2  # 16-bar idea
time_step = 360

for note in melody_notes:
    track.append(Message('note_on', note=note, velocity=90, time=time_step))
    track.append(Message('note_off', note=note, velocity=90, time=240))

mid.save("midi-clips/melody.mid")


## 4. Play MIDI with pygame

In [ ]:

# !pip install pygame

In [ ]:
import pygame

pygame.mixer.init()
pygame.mixer.music.load("midi-clips/melody.mid")
pygame.mixer.music.play()

print("Playing MIDI...")
while pygame.mixer.music.get_busy():
    pass


Playing MIDI...


## 5. MIDI to Audio 

In [ ]:
import os
import subprocess

sf = os.path.expanduser("~/Downloads/FluidR3GM2-2.SF2")

clip = "melody"

midi = "midi-clips/"+clip+".mid"
out = "audio-clips/"+clip+".wav"

cmd = [
    "fluidsynth",
    "-n",             # no audio driver
    "-i",             # no shell
    "-F", out,        # write to file
    "-r", "44100",    # sample rate
    sf,               # SoundFont
    midi              # MIDI file
]

print("Running:", " ".join(cmd))
subprocess.run(cmd)
print("Saved:", out)


Running: fluidsynth -n -i -F audio-clips/melody.wav -r 44100 /Users/rishik.marseni/Downloads/FluidR3GM2-2.SF2 midi-clips/melody.mid
FluidSynth runtime version 2.5.1
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'audio-clips/melody.wav'..
Saved: audio-clips/melody.wav


## PyDub

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

# Load pre-rendered clips
beat = AudioSegment.from_wav("audio-clips/beat.wav")          # short loop
harmony = AudioSegment.from_wav("audio-clips/harmony.wav")    # short loop
melody = AudioSegment.from_wav("audio-clips/melody.wav")      # long single clip

# Repeat beat & harmony to match melody length
repeat_count = int(len(melody) / len(beat)) + 1
beat_loop = beat * repeat_count
harmony_loop = harmony * repeat_count

# Trim loops to melody length
beat_loop = beat_loop[:len(melody)]
harmony_loop = harmony_loop[:len(melody)]

# Mix all tracks
final_mix = beat_loop.overlay(harmony_loop).overlay(melody)

# Play
# play(final_mix)

# export
final_mix.export("audio-clips/final_song_pydub.wav", format="wav")


<_io.BufferedRandom name='audio-clips/final_song_pydub.wav'>

In [ ]:
# !pip install midi2audio
# ! pip install pygame

## Librosa

In [ ]:
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio

# Load audio as numpy arrays
beat, sr = librosa.load("audio-clips/beat.wav", sr=None)
harmony, _ = librosa.load("audio-clips/harmony.wav", sr=None)
melody, _ = librosa.load("audio-clips/melody.wav", sr=None)

# Repeat beat & harmony to match melody length
repeat_count = int(len(melody) / len(beat)) + 1
beat_loop = np.tile(beat, repeat_count)[:len(melody)]
harmony_loop = np.tile(harmony, repeat_count)[:len(melody)]

# Mix tracks
final_mix = beat_loop + harmony_loop + melody

# Normalize to prevent clipping
final_mix = final_mix / np.max(np.abs(final_mix))

# Save final audio
sf.write("audio-clips/final_song_librosa.wav", final_mix, sr)

# Play in notebook
Audio(final_mix, rate=sr)
